In [1]:
import nltk
from nltk.tokenize import RegexpTokenizer
import numpy as np
from collections import Counter
# from nltk.corpus import stopwords
# nltk.download('stopwords')

In [2]:
# stop_words = set(stopwords.words('english'))
austen = nltk.corpus.gutenberg.words('austen-sense.txt')

# Using Jane Austen's **Sense**
## [1] Preprocessing
* Remove all non-words
* ~~Remove all stop words~~
* Lowercase

I looked into removing stop words, but using NLTK.corpus.stopwords reduced the total number of tokens to ~58k, too low per the instructions. Stemming and lemmatizing would further reduce this.

In [3]:
austen = ' '.join(austen)
tokenizer = RegexpTokenizer(r'\w+')
austen = tokenizer.tokenize(austen) # remove nonwords
# austen = [word for word in austen if not word in stop_words] # remove stop words
austen_list = [word.lower() for word in austen] # lowercase
austen_str = ' '.join(austen)

### [1.1] Word-to-Int mapping

In [4]:
freqs = Counter()
tokens = austen_str.lower().split()
for token in tokens:
    freqs[token] += 1                                 # freqs is dict(), i.e. freqs[word] = count
freqs = freqs.most_common()                   # now freqs is tuple of (word, count)
mapping = {freqs[i][0]:i+1 for i in range(len(freqs))}  # mapping is dict of freqs[word]=r where r=rank

### [1.2] Split the corpus into documents on the word "chapter"

In [5]:
docs = []
start = 0
i = 0
while i < len(austen_list):
    if austen_list[i] == 'chapter':
        docs.append(austen_list[start:i-1])
        start = i
    i += 1
    
docs = docs[1:]
N_W = sum(len(doc) for doc in docs)
N_W_unique = len(set(austen_list))
N_D = len(docs)
print('Total number of tokens: ' + str(N_W))
print('Number of unique tokens: ' + str(N_W_unique))
print('Number of documents: ' + str(N_D))

Total number of tokens: 118236
Number of unique tokens: 6336
Number of documents: 49


### [1.3] Create word-to-int mapped matrix
The rows of the matrix will be documents and 

### Collapsed Gibbs Sampling
$\alpha$ = $\beta$ = 0.01

In [23]:
N_K = 10
T = 10
alpha = 0.01
beta = alpha

#### Initialization

In [ ]:
ndw = np.random.randint(low=0, high=N_K, size=(N_D, N_W)) # Z
ndk = np.zeros((N_D, N_K)) # Pi
nkw = np.zeros((N_K, N_W)) # B
for i in range(N_D):
    ndk[i] = np.random.dirichlet(alpha * np.ones(N_K))
for i in range(N_K):
    nkw[i] = np.random.dirichlet(beta * np.ones(N_W))

#### Iterate

In [19]:
for t in range(T):
    for i in range(N_D):
        for j in range(N_W):
            p_ij = np.exp(np.log(ndk[i]) + np.log(nkw[:, ]))

array([1.31122896e-031, 1.15679443e-007, 1.59470696e-010, 1.20598730e-102,
       2.87670449e-044, 3.99890367e-009, 2.21264749e-061, 1.65514186e-069,
       9.99996581e-001, 3.29923486e-006])

In [14]:
nkw.shape

NameError: name 'nkw' is not defined

In [8]:
np.log(0)

<ipython-input-8-f6e7c0610b57>:1: RuntimeWarning: divide by zero encountered in log
  np.log(0)


-inf